In [65]:
import requests
import time
from bs4 import BeautifulSoup
from multiprocessing.pool import ThreadPool

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [66]:
def get_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome('../bins/chromedriver', options = options)
    return driver
#https://chromedriver.storage.googleapis.com/95.0.4638.69/chromedriver_mac64_m1.zip
#https://chromedriver.storage.googleapis.com/95.0.4638.69/chromedriver_linux64.zip

In [75]:
def get_paths(path):
    driver = get_driver()
    driver.get(path)
    
    match_paths = list(map(lambda x: x.get_dom_attribute('href'), 
                           driver.find_elements(By.CLASS_NAME, 'ind_match_wrapper')))
    match_paths = match_paths[1:]
    match_paths = list(map(lambda x: 'https://www.xscores.com/' + x, match_paths))
    teams = list(map(lambda x: x.split('/')[-3].split('-vs-'), match_paths))
    
    driver.close()
    driver.quit()
    return match_paths, teams

# def get_stats(paths):
#     for match_path in paths:
#         driver = get_driver()
#         driver.get(match_path)
        
#         driver.close()
#         driver.quit()
    
def get_info(path):
    driver = get_driver()
    driver.get(path)
        
    date = driver.find_element(By.CLASS_NAME, 'match_details_date').text
    bets = driver.find_element(By.CLASS_NAME, 'mDetails-odds').text.split('\n')[1::2]
        
    driver.close()
    driver.quit()
    return date, bets
    
def get_stats(paths):
    with ThreadPool(40) as p:
        results = p.map(get_info, paths)
    return results

In [76]:
path = 'https://www.xscores.com/soccer/england/premier-league'
paths, teams = get_paths(path)
start_time = time.time()
stats = get_stats(paths)
data = list(zip(teams, stats))
print("--- %s seconds ---" % (time.time() - start_time))

/var/folders/lm/flgwr_ks0cn6xv9bcn3f_d2m0000gn/T/ipykernel_72511/1996232620.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../bins/chromedriver', options = options)


--- 4.7816948890686035 seconds ---


In [81]:
(homeTeam, AwayTeam), (date, bets) = data[0]

In [89]:
data[0][0][0].title()

'Leicester'